# Full Final Code

In [1]:
import pandas as pd
import requests
import numpy as np
import json
from ast import literal_eval
from pandas.io.json import json_normalize
from datetime import datetime
import re

Cleaning movies_metadata || Completed by Lea

In [2]:
# Data Source 1 - movies_metadata 

movie_data = "movies_metadata.csv"
movies_df = pd.read_csv(movie_data)
movies_df.columns

C:\Users\Lealo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [3]:
# Remove Unwanted Columns

movies_df = movies_df.drop(['adult', 'belongs_to_collection', 'genres', 'homepage', 'original_language',
                'original_title', 'overview', 'popularity', 'poster_path', 'production_countries',
                'runtime', 'spoken_languages', 'status', 'tagline', 'video', 'vote_count'], axis=1)

In [4]:
# Rename Columns - NOTE: Haven't actually processed this one because our individual code might rely on specific naming. 

# Non-permanent code:
##movies_df.rename(columns={'budget': 'production_budget', 'id': 'movie_id', 'title': 'movie_name', 'vote_average': 'imdb_rating'})

# Permanent code:
movies_df = movies_df.rename(columns={'budget': 'production_budget', 'id': 'movie_id', 'title': 'movie_name', 
                                      'vote_average': 'imdb_rating'})


Adding MovieLens Ratings to movies_df || Completed by Lea

In [5]:
# Drop all blank movie ID's

movies_df = movies_df.dropna(subset=['movie_id'])

In [6]:
# Removing the 0's before movie_id 

movies_df['movie_id'] = movies_df['movie_id'].astype(str).str.replace('\D+', '')

In [7]:
# Setting movie_id to int

movies_df['movie_id'] = movies_df['movie_id'].apply(lambda x: int(x))

In [8]:
# Data Source 2 - ratings 

ratings_data = "ratings.csv"
ratings_df = pd.read_csv(ratings_data)

In [9]:
ratings_df = ratings_df.groupby(['movieId'], as_index = False).mean()

In [10]:
# Add rating to movies_df

movies_df['rating'] = movies_df.movie_id.map(ratings_df.set_index('movieId')['rating'].to_dict())

Production Company Parsing || Completed by Brian

In [11]:
csv_file = "movies_metadata.csv"

# df = pd.read_csv(csv_file,'unicode_escape')

df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape')

# csv_file.encode('utf-8').strip(csv_file)

In [12]:
# Step 2: Isolation of columns that are relevant
movie_genres = df[['id', 'genres']]

In [14]:
prod_company = df[["id", "production_companies"]]

In [16]:
parse_companies_df = pd.DataFrame()

for row_number, movie in prod_company.iterrows():
    company_list = []
    try:
        for company_counter, company in enumerate(literal_eval(movie['production_companies'])):
            company_list.append(company.get('name'))
    except ValueError:
        pass
        
    parse_companies_df = parse_companies_df.append({'id': movie['id'],
                                                   'production_companies': ', '.join(map(str, company_list))},
                                           ignore_index=True,
                                           sort=False)

TypeError: 'bool' object is not iterable

In [ ]:
df.drop(['genres', 'production_companies'], axis=1, inplace = True)

In [ ]:
df = df.merge(parse_companies_df, on="id")

In [ ]:
df = df.merge(parse_genre_df, on="id")

In [ ]:
df.to_csv('new_movie_metadata.csv', index=False)

Yearly Production Studio Spend || Completed by Brian

In [ ]:
csv_file = "new_movie_metadata.csv"

df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape')

In [ ]:
release_date = df[['id','release_date']].head()

In [ ]:
#a for loop to pick up dates in the excel that are messy.
year_list = []

for row_number, date in df["release_date"].items():
    if isinstance(date, float) or len(date) <=3:
        year_list.append(None)
    else:
        try:
            date_year = datetime.strptime(date, "%m/%d/%Y")
            year_list.append(date_year.year)
        except ValueError:
            date_year = datetime.strptime(date, "%Y-%m-%d")
            year_list.append(date_year.year)

In [ ]:
#new column, who dis?
df["release_year"] = year_list

In [ ]:
df.columns

In [ ]:
df[["release_date","release_year"]]

In [ ]:
production = df[["production_companies","revenue","budget","title","release_year"]].copy()

In [ ]:
production = production.dropna(subset=['title','production_companies'])

In [ ]:
top = production.sort_values(by = ["revenue"],ascending = False)

In [ ]:
#filtering production companies with Marvel in string
Marvel = production[production['production_companies'].str.contains('Marvel')]

In [ ]:
#looking at the most recent film in the table
sort_year = Marvel.sort_values(by = ["release_year"],ascending = False)

In [ ]:
#sorting by title since this table didn't have an index prior. dropped films that had 0 info
top_ten = sort_year.set_index('title').drop(['Team Thor','Marvel Studios: Assembling a Universe',
                                             'Marvel One-Shot: All Hail the King',
                                             'Iron Man & Captain America: Heroes United',
                                             'Marvel: 75 Years, From Pulp to Pop!','Iron Man & Hulk: Heroes United',
                                             'Marvel One-Shot: Agent Carter','Marvel One-Shot: Item 47',
                                             'Thor: Tales of Asgard','Planet Hulk'])

In [ ]:
top_ten.head(57)

In [ ]:
#reseting the index
top_ten.reset_index(level='title')

In [ ]:
#convert budget to float since sum below wouldnt work as a object
top_ten["budget"] = top_ten["budget"].astype(float)

In [ ]:
revenue_sum = top_ten.groupby(['release_year'])['revenue'].sum()

budget_sum = top_ten.groupby(['release_year'])['budget'].sum()

In [ ]:
merge_inner = pd.merge(left=revenue_sum, right=budget_sum, left_on='release_year', right_on='release_year')

In [ ]:
#final table with total revenue and budget per year
merge_inner.sort_values(by = ["release_year"],ascending = False)

Franchise Code || Completed by Kirsten

In [ ]:
#Pull website data and extract table
url = 'https://www.the-numbers.com/movies/franchises'

response = requests.get(url)

tables = pd.read_html(response.text)

f_all_data_df = tables[0]

In [ ]:
#Keep relevant rows: Franchise name and Worldwide Box Office
franchise_df = f_all_data_df[["Franchise", "Worldwide Box Office"]]

In [ ]:
#Remove currency formatting and drop franchises with no value or 0  
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace(',', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace('$', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].astype(float)

franchise_df.replace(0, np.nan, inplace=True)
franchise_df.dropna(inplace=True)

In [ ]:
#Export franchise data
#franchise_df.to_csv('franchise_df.csv', index=False)

In [ ]:
#Read metadata CSV
csv_file = "new_movie_metadata.csv"

metadata_df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape', low_memory=False)

In [ ]:
#Keep relevant rows: original_title, collection, revenue. Drop rows with 0 value
franchise_single = metadata_df[['original_title', 'belongs_to_collection', 'revenue']]

franchise_single.astype({'revenue': 'float64'}).dtypes

franchise_single = franchise_single.query("revenue > 0")
franchise_single = franchise_single.reset_index(drop=True)
franchise_single.dropna(inplace=True)

In [ ]:
#Export data for individual movies
#franchise_single.to_csv('metadata_franchise.csv', index=False)

In [ ]:
#Create empty dataframe for final table
final_table = pd.DataFrame (columns = ['Franchise', 'Movie Title','Movie Revenue', 
                                       'Total Franchise Revenue', '% of Total Franchise Revenue'])

In [ ]:
#Match if collection contains franchise name and input desired data into final table
for idx, fn in franchise_df.iterrows():
    search = fn["Franchise"]
    wwbo = fn["Worldwide Box Office"]
    title_match = franchise_single["belongs_to_collection"].str.contains(search)
    indexes = (title_match[title_match==True].index.values)
    
    revenue = franchise_single.loc[indexes, 'revenue'] 
    movie_title = franchise_single.loc[indexes, 'original_title'] 
    franchise_performance = (revenue / wwbo)*100

    final_data = {'Franchise': search, 'Movie Title': movie_title, 'Movie Revenue': revenue, 
                  'Total Franchise Revenue': wwbo, '% of Total Franchise Revenue': franchise_performance}
    final_table = final_table.append(pd.DataFrame(final_data, columns= ['Franchise', 'Movie Title', 
                                                                        'Movie Revenue', 'Total Franchise Revenue', 
                                                                        '% of Total Franchise Revenue']),ignore_index=True)

In [ ]:
#Clean final table
delete='[REC]'

clean_final = final_table[final_table.Franchise != delete]

#Export final table to CSV
clean_final.to_csv('Franchise_Performance_By_Movie.csv', index=False)